Часть 1. Написание SQL-запросов
В базах hh.ru есть различные данные о вакансиях и резюме. Пример того, как эти данные устроены, можно посмотреть в excel-файле (hh_аналитик_таблицы_01): там есть и описание данных в том числе. Изучите этот файл и на основе таблиц, указанных в примере, напишите 3 sql-запроса (в идеале на диалектах presto или hive), которые «достанут» из этих таблиц нужные срезы данных:
1.     Выгрузить число созданных вакансий (в динамике по месяцам), опубликованных в России, в названии которых встречается слово «водитель», предлагающих «Гибкий график» работы, за 2020-2021 годы. Важно, чтобы вакансии на момент сбора данных не были удаленными / заблокированными.
2.     Выяснить, в каких регионах РФ (85 штук) выше всего доля вакансий, предлагающих удаленную работу. Вакансии должны быть не заархивированными, не заблокированными и не удаленными, и быть созданными в 2021-2022 годах.
3.     Подсчитать «вилку» (10,25,50,75 и 90 процентиль) ожидаемых зарплат (в рублях) из московских и питерских резюме, имеющих роль «разработчик» (id роли — 91), по городам и возрастным группам (группы сделать как в примере таблицы ниже, не учитывать резюме без указания даты рождения — такие тоже есть). Возрастные группы считать на дату составления запроса. Резюме должно быть не удалено и иметь статус «завершено». Дополнительно выяснить (при помощи того же запроса) долю резюме по каждой группе, в которых указана ожидаемая зарплата. Пример таблицы, которая должна получиться на выходе:


Создадим необходимые таблицы

In [ ]:
#импортируем библиотеки
import numpy as np
import pandas as pd
import random
import sqlite3
import datetime

In [ ]:
vacancy_data = [
    {'vacancy_id': 1, 'name': 'Водитель', 'work_schedule': 'Гибкий график', 'disabled': False, 'area_id': 1, 'creation_time': '1/3/20 14:02', 'archived': False},
    {'vacancy_id': 2, 'name': 'Аналитик', 'work_schedule': 'Полный день', 'disabled': False, 'area_id': 2, 'creation_time': '2/10/21 13:13', 'archived': False},
    {'vacancy_id': 3, 'name': 'Маркетолог', 'work_schedule': 'Удаленная работа', 'disabled': False, 'area_id': 13, 'creation_time': '2/1/22 11:25', 'archived': False}
]

resume_data = [
    {'resume_id': 1, 'disabled': False, 'is_finished': 1, 'area_id': 1, 'compensation': 300, 'currency': 'EUR', 'birth_day': '10/13/75 0:00', 'role_id_list': '1, 13, 4,91'},
    {'resume_id': 2, 'disabled': False, 'is_finished': 1, 'area_id': 2, 'compensation': 100000, 'currency': 'RUR', 'birth_day': '5/21/93 0:00', 'role_id_list': '2, 14,91'},
    {'resume_id': 3, 'disabled': True, 'is_finished': 3, 'area_id': 4, 'compensation': 1500, 'currency': 'USD', 'birth_day': '12/12/87 0:00', 'role_id_list': '91'},
    {'resume_id': 4, 'disabled': False, 'is_finished': 1, 'area_id': 1, 'compensation': 7000, 'currency': 'EUR', 'birth_day': '10/13/75 0:00', 'role_id_list': '1, 13, 4,91'}
]

area_data = [
    {'area_id': 1, 'area_name': 'Москва', 'region_name': 'Москва', 'country_name': 'Россия'},
    {'area_id': 2, 'area_name': 'Санкт-Петербург', 'region_name': 'Санкт-Петербург', 'country_name': 'Россия'},
    {'area_id': 3, 'area_name': 'Новосибирск', 'region_name': 'Новосибирская область', 'country_name': 'Россия'},
    {'area_id': 13, 'area_name': 'Саранск', 'region_name': 'Республика Мордовия', 'country_name': 'Россия'}
]

currency_data = [
    {'code': 'RUR', 'rate': 1},
    {'code': 'USD', 'rate': 0.013074},
    {'code': 'EUR', 'rate': 0.01159}
]

# Создаем DataFrame
df_vacancy = pd.DataFrame(vacancy_data)
df_resume = pd.DataFrame(resume_data)
df_area = pd.DataFrame(area_data)
df_currency = pd.DataFrame(currency_data)



*В процессе тестирования запросов я менял исходный шаблон данных.*

In [ ]:
#создание бд и заливка
con = sqlite3.connect('db')
df_vacancy.to_sql('vacancy',con,index=False,if_exists='replace')
df_resume.to_sql('resume',con,index=False,if_exists='replace')
df_area.to_sql('area',con,index=False,if_exists='replace')
df_currency.to_sql('currency',con,index=False,if_exists='replace')

3

In [ ]:
#1
sql = ''' SELECT  CAST(SUBSTR(creation_time, INSTR(creation_time, '/')+1) - SUBSTR(creation_time, INSTR(creation_time, '/')) AS TEXT)  || '-' ||
  CAST(SUBSTR(SUBSTR(creation_time, INSTR(creation_time, ' ')-2), 1, 2) AS TEXT) AS mY,
COUNT(*) AS count
FROM vacancy
JOIN area ON
vacancy.area_id = area.area_id
WHERE
    vacancy.work_schedule = 'Гибкий график'
    AND vacancy.disabled = False
    AND vacancy.name LIKE '%Водитель%'
    AND SUBSTR(SUBSTR(creation_time, INSTR(creation_time, ' ')-2), 1, 2) in ('20','21')
GROUP BY
  SUBSTR(creation_time, INSTR(creation_time, '/')+1) - SUBSTR(creation_time, INSTR(creation_time, '/')), SUBSTR(SUBSTR(creation_time, INSTR(creation_time, ' ')-2), 1, 2)
ORDER BY
  SUBSTR(creation_time, INSTR(creation_time, '/')+1) - SUBSTR(creation_time, INSTR(creation_time, '/')), SUBSTR(SUBSTR(creation_time, INSTR(creation_time, ' ')-2), 1, 2);

'''

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
select(sql)

,mY,count
0,3-20,1


Проверка,что есть такая вакансия с датой создания 1/3/20 14:02 и она одна

In [ ]:
#2
sql = ''' SELECT  area.region_name as region,
COUNT(CASE WHEN vacancy.work_schedule = 'Удаленная работа' THEN 1 END) * 100.0 / COUNT(*) AS remote_share
FROM vacancy
JOIN area ON
vacancy.area_id = area.area_id
WHERE
    vacancy.disabled = False
    AND vacancy.archived = False
    AND SUBSTR(SUBSTR(creation_time, INSTR(creation_time, ' ')-2), 1, 2) in ('22','21')

GROUP BY
area.region_name
ORDER BY
COUNT(CASE WHEN vacancy.work_schedule = 'Удаленная работа' THEN 1 END) * 100.0 / COUNT(*) DESC;

'''

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
select(sql)

,region,remote_share
0,Республика Мордовия,100.0
1,Санкт-Петербург,0.0


Проверка.У нас 1 вакансия удаленная вакансия Маркетолога в Республике Мордовия.

In [ ]:
#3
sql =''' WITH ConvertedCompensations AS (
    SELECT
        area.region_name AS Город,
        CASE
            WHEN CAST(strftime('%Y', 'now') AS INTEGER) - CAST('19' || CAST(SUBSTR(SUBSTR(r.birth_day, INSTR(r.birth_day, ' ')-2), 1, 2) AS TEXT) AS INTEGER) <= 17 THEN '17 лет и младше'
            WHEN CAST(strftime('%Y', 'now') AS INTEGER) - CAST('19' || CAST(SUBSTR(SUBSTR(r.birth_day, INSTR(r.birth_day, ' ')-2), 1, 2) AS TEXT) AS INTEGER) BETWEEN 18 AND 24 THEN '18–24'
            WHEN CAST(strftime('%Y', 'now') AS INTEGER) - CAST('19' || CAST(SUBSTR(SUBSTR(r.birth_day, INSTR(r.birth_day, ' ')-2), 1, 2) AS TEXT) AS INTEGER) BETWEEN 25 AND 34 THEN '25–34'
            WHEN CAST(strftime('%Y', 'now') AS INTEGER) - CAST('19' || CAST(SUBSTR(SUBSTR(r.birth_day, INSTR(r.birth_day, ' ')-2), 1, 2) AS TEXT) AS INTEGER) BETWEEN 35 AND 44 THEN '35–44'
            WHEN CAST(strftime('%Y', 'now') AS INTEGER) - CAST('19' || CAST(SUBSTR(SUBSTR(r.birth_day, INSTR(r.birth_day, ' ')-2), 1, 2) AS TEXT) AS INTEGER) BETWEEN 45 AND 54 THEN '45–54'
        ELSE '55 и старше'
        END AS Возрастная_группа,

        CAST(r.compensation AS REAL) * (1 / c.rate) AS numeric_compensation
    FROM resume r
    JOIN area ON r.area_id = area.area_id
    JOIN currency c ON r.currency = c.code
    WHERE r.birth_day IS NOT NULL
    AND (INSTR(r.role_id_list, ',91,') > 0 OR INSTR(r.role_id_list, ',91') > 0 OR INSTR(r.role_id_list, '91,') > 0 OR r.role_id_list = '91')
    AND area.area_id IN ('1', '2')
    AND r.disabled = False
    AND r.is_finished = '1'
),
RankedCompensations AS (
    SELECT
        Город,
        Возрастная_группа,
        numeric_compensation,
        ROW_NUMBER() OVER (PARTITION BY Город, Возрастная_группа ORDER BY numeric_compensation) AS rn,
        COUNT(*) OVER (PARTITION BY Город, Возрастная_группа ORDER BY numeric_compensation) AS total
    FROM ConvertedCompensations
)
SELECT
    Город,
    Возрастная_группа,
    MAX(CASE WHEN rn = CEIL(total * 0.1) THEN numeric_compensation END) AS p10,
    MAX(CASE WHEN rn = CEIL(total * 0.25) THEN numeric_compensation END) AS p25,
    MAX(CASE WHEN rn = CEIL(total * 0.5) THEN numeric_compensation END) AS p50,
    MAX(CASE WHEN rn = CEIL(total * 0.75) THEN numeric_compensation END) AS p75,
    MAX(CASE WHEN rn = CEIL(total * 0.9) THEN numeric_compensation END) AS p90,
    (COUNT(CASE WHEN numeric_compensation IS NOT NULL THEN 1 ELSE NULL END) * 100.0) / COUNT(*) AS 'Доля резюме с указанной зарплатой'
FROM RankedCompensations
GROUP BY Город, Возрастная_группа;

'''

In [ ]:
def select(sql):
  return pd.read_sql(sql,con)

In [ ]:
select(sql)

,Город,Возрастная_группа,p10,p25,p50,p75,p90,Доля резюме с указанной зарплатой
0,Москва,45–54,25884.383089,25884.383089,25884.383089,603968.93874,603968.93874,100.0
1,Санкт-Петербург,25–34,100000.000000,100000.000000,100000.000000,100000.00000,100000.00000,100.0


Я не уверен,что это верно так как p10 	p25 дублируються. Даже chat gpt зависает от таких запросов.

Часть 2. Визуализация данных и выводы
У вас есть набор таблиц (файл hh_аналитик_таблицы_02) с данными о вакансиях и резюме на тему командировок. Сделайте дашборд в tableau public с визуализацией этих данных и выводами.


К сожалению не удалось построить дешборд в tableau по техническим причинам. Табло паблик требует более новую виндовс. А веб версия, как я понял имеет ограничения на подключения к таблицам и не дает создать дешборд. У меня получилось только создать листы. [1](https://public.tableau.com/views/sheet1_17287656796630/Sheet1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link) [2](https://public.tableau.com/views/sheet2_17287665190190/Sheet1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link) [3](https://public.tableau.com/views/sheet3_17287667864280/Sheet1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)  [4](https://public.tableau.com/views/sheet4_17287702196930/Sheet1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link) [5](https://public.tableau.com/views/sheet5_17287688866390/Sheet1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link) [6](https://public.tableau.com/views/sheet6_17287691906860/Sheet1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link) . Признаюсь, не являюсь уверенным в tableau,только начинающий,но есть небольшая практика создания простых дешбордов в разных BI (в т.ч.tableau). Доступно тут https://drive.google.com/drive/folders/1lqzk3doPZtDixv77weycAWaNIs0Y9AiV

Выводы:


*   Доля вакансий с упоминанием обязательных командировок в 2021 году незначительно снизилась по сравнению с 2019 годом,с 1,5% до 1,3%.

*   Доля вакансий за 2019,2020,2021 годы с упоминанием обязательных командировок в секторах:химическое производство,элетроника и приборостроение,энергетика остается стабильно выше,по сравнению другими секторами.


*   Предлогаемая медианная зарплата в 2019,2020,2021 годах была выше у вакансий с командировками.


*   Более половины (55%) в резюме по статусам готовности к командировкам указали,что не готовы к командировка.Готовы к командировкам в статусе резюме указали 30%.



*   
Больше всего в 2021 году среди профобластей в резюме указали готовность к командировкам:добыча сырья (64%),высший менеджемент (62%).Готовность к редким командировкам выше:управления персоналом (23%) и бухгалтерия,упр.учет (23%).Не готовность к командировкам выше: начало карьеры,студенты (82%) и домашнего персонала (67%).


*   В 2021 году больше всего указали в статусе готовность к командировке-мужчины (38%) и категория в возрасте 35-44 лет(38%).Готовы к редким командировкам больше женщины (15%) и категория в возрасте 35-44 лет(19,57%).Не готовы к командировкам больше женщины (61%) и категория 17 и младше лет  (98%).



email:makarenkomyinbox@gmail.com
Tg:@Vyacheslav0